In [1]:
import pydicom
from PIL import Image
import numpy as np
import pandas as pd
import os

In [3]:
base_path = "/mnt/c/Users/caspe/Thesis-Synthex/"
data_path = "data/LIDC-IDRI-raw/"
output_path = "data/LIDC-IDRI-XRAY/"
files = os.listdir(base_path + data_path)
df_metadata = pd.read_csv(base_path + data_path + [file for file in files if file.endswith(".csv")][-1])



In [42]:
series_id = df_metadata[(df_metadata["PatientID"] == "LIDC-IDRI-0438") & (df_metadata["Modality"] == "SEG")].iloc[0]["SeriesInstanceUID"]

In [43]:
base_path + data_path + series_id +"/1-1.dcm"

'/mnt/c/Users/caspe/Thesis-Synthex/data/LIDC-IDRI-raw/1.2.276.0.7230010.3.1.3.0.28122.1553311600.906021/1-1.dcm'

In [49]:
dataset = pydicom.dcmread(base_path + data_path + series_id +"/1-1.dcm")

In [46]:
dataset.convert_pixel_data("numpy")

In [59]:
print(dataset.pixel_array.sum())

dataset.pixel_array.shape

812


(13, 512, 512)

In [55]:
dataset.pixel_array.astype(np.float32)

1

In [56]:
import meshlib.mrmeshpy as mr
import meshlib.mrmeshnumpy as mrn
#convert 3D array to SimpleVolume data
simpleVolume = mrn.simpleVolumeFrom3Darray(dataset.pixel_array.astype(np.float32))

#convert SimpleVolume to FloatGrid data
floatGrid = mr.simpleVolumeToDenseGrid(simpleVolume )

#make mesh by iso-value = 0.5 and voxel size = (0.1, 0.1, 0.1)
mesh = mr.gridToMesh(floatGrid , mr.Vector3f(0.1, 0.1, 0.1), 0.5)

#save mesh
mr.saveMesh(mesh, "/mnt/c/Users/caspe/Desktop/nodule.stl") 

In [19]:
rad = pd.read_csv("/mnt/c/Users/caspe/Thesis-Synthex/radiologist_experiment.csv", sep=";")

In [31]:
rad_lidc = rad[rad["path"].str.contains("LIDC")]

In [38]:
rad_lidc["TCIA Patent ID"] = rad_lidc["path"].apply(lambda x : "LIDC-IDRI-" + x.split("-")[-1].split(".")[0])

/tmp/ipykernel_3552/297623319.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rad_lidc["TCIA Patent ID"] = rad_lidc["path"].apply(lambda x : "LIDC-IDRI-" + x.split("-")[-1].split(".")[0])


In [30]:
ids = rad[rad["path"].str.contains("LIDC")]["path"].apply(lambda x : "LIDC-IDRI-" + x.split("-")[-1].split(".")[0])

In [40]:
rad_lidc.columns

Index(['Unnamed: 0', 'level_0', 'index', 'path', 'id', 'tobias_answer_q1',
       'tobias_answer_q2', 'samir_answer_q1', 'samir_answer_q2',
       'jack_answer_q1', 'jack_answer_q2', 'tobias_tekstsvar',
       'samir_tekstsvar', 'jack_tekstsvar', 'TCIA Patient ID',
       'TCIA Patent ID'],
      dtype='object')

In [27]:
nodule_sizes = pd.read_excel("/mnt/c/Users/caspe/Thesis-Synthex/lidc-idri-nodule-counts-6-23-2015.xlsx")

In [37]:
nodule_sizes.columns

Index(['TCIA Patent ID', 'Total Number of Nodules* ',
       'Number of Nodules >=3mm**', 'Number of Nodules <3mm***', 'Unnamed: 4',
       'Unnamed: 5'],
      dtype='object')

In [62]:
rad_lidc_nodules = rad_lidc.merge(nodule_sizes, how="inner", on= "TCIA Patent ID")[ ['TCIA Patent ID',"path",

       'tobias_answer_q2',  'samir_answer_q2',
      'jack_answer_q2',
    
    'Total Number of Nodules* ',
       'Number of Nodules >=3mm**', 'Number of Nodules <3mm***', 'Unnamed: 4',
       'Unnamed: 5']]

In [64]:
rad_lidc_nodules["type"] = rad_lidc_nodules["path"].apply(lambda x: x.split("-")[2])

In [68]:
rad_lidc_nodules = rad_lidc_nodules[['TCIA Patent ID', 'type', 'tobias_answer_q2', 'samir_answer_q2',
       'jack_answer_q2', 'Total Number of Nodules* ',
       'Number of Nodules >=3mm**', 'Number of Nodules <3mm***']]

In [72]:
rad_lidc_nodules.columns = ['Patient ID', 'type', 'rad1_q2', 'rad2_q2',
       'rad3_q2', 'N Nodules* ',
       ' N Nodules >=3mm**', 'N Nodules <3mm***']

In [79]:
print(rad_lidc_nodules.sort_values("Patient ID").to_latex(index=False))

\begin{tabular}{lllllrrr}
\toprule
Patient ID & type & rad1_q2 & rad2_q2 & rad3_q2 & N Nodules*  &  N Nodules >=3mm** & N Nodules <3mm*** \\
\midrule
LIDC-IDRI-0001 & XRAY & ['Lung nodules'] & ['Lung nodules'] & ['Lung nodules'] & 4 & 1 & 3 \\
LIDC-IDRI-0003 & XRAY & ['Lung nodules'] & ['Lung nodules'] & ['Lung nodules'] & 4 & 4 & 0 \\
LIDC-IDRI-0005 & XRAY & ['None'] & ['None'] & ['None'] & 9 & 3 & 6 \\
LIDC-IDRI-0006 & XRAY & ['None'] & ['Lung nodules'] & ['Covid-19 lesions'] & 14 & 4 & 10 \\
LIDC-IDRI-0012 & XRAY & ['Lung nodules'] & ['Lung nodules'] & ['Lung nodules'] & 16 & 12 & 4 \\
LIDC-IDRI-0022 & XRAY & ['Lung nodules'] & ['Both'] & ['Lung nodules'] & 4 & 1 & 3 \\
LIDC-IDRI-0023 & XRAY & ['None'] & ['None'] & ['None'] & 2 & 1 & 1 \\
LIDC-IDRI-0037 & XRAY & ['Lung nodules'] & ['Covid-19 lesions'] & ['Covid-19 lesions'] & 3 & 2 & 1 \\
LIDC-IDRI-0040 & XRAY & ['Covid-19 lesions'] & ['Covid-19 lesions'] & ['Covid-19 lesions'] & 8 & 4 & 4 \\
LIDC-IDRI-0042 & XRAY & ['None'] & ['Non

In [80]:
rad_lidc_nodules.sort_values("Patient ID")

,Patient ID,type,rad1_q2,rad2_q2,rad3_q2,N Nodules*,N Nodules >=3mm**,N Nodules <3mm***
17,LIDC-IDRI-0001,XRAY,['Lung nodules'],['Lung nodules'],['Lung nodules'],4,1,3
7,LIDC-IDRI-0003,XRAY,['Lung nodules'],['Lung nodules'],['Lung nodules'],4,4,0
11,LIDC-IDRI-0005,XRAY,['None'],['None'],['None'],9,3,6
14,LIDC-IDRI-0006,XRAY,['None'],['Lung nodules'],['Covid-19 lesions'],14,4,10
4,LIDC-IDRI-0012,XRAY,['Lung nodules'],['Lung nodules'],['Lung nodules'],16,12,4
18,LIDC-IDRI-0022,XRAY,['Lung nodules'],['Both'],['Lung nodules'],4,1,3
5,LIDC-IDRI-0023,XRAY,['None'],['None'],['None'],2,1,1
3,LIDC-IDRI-0037,XRAY,['Lung nodules'],['Covid-19 lesions'],['Covid-19 lesions'],3,2,1
16,LIDC-IDRI-0040,XRAY,['Covid-19 lesions'],['Covid-19 lesions'],['Covid-19 lesions'],8,4,4
15,LIDC-IDRI-0042,XRAY,['None'],['None'],['None'],38,10,28


In [61]:
rad_lidc.merge(nodule_sizes, how="inner", on= "TCIA Patent ID")[ ['TCIA Patent ID',"path",

       'tobias_answer_q2',  'samir_answer_q2',
      'jack_answer_q2',
    
    'Total Number of Nodules* ',
       'Number of Nodules >=3mm**', 'Number of Nodules <3mm***', 'Unnamed: 4',
       'Unnamed: 5']]

,TCIA Patent ID,path,tobias_answer_q2,samir_answer_q2,jack_answer_q2,Total Number of Nodules*,Number of Nodules >=3mm**,Number of Nodules <3mm***,Unnamed: 4,Unnamed: 5
0,LIDC-IDRI-0876,LIDC-IDRI-DRR-CROPPEDLIDC-IDRI-0876.png,['None'],['None'],['None'],2,0,2,NaN,NaN
1,LIDC-IDRI-0438,LIDC-IDRI-DRR-CROPPEDLIDC-IDRI-0438.png,['None'],['None'],['None'],5,1,4,NaN,NaN
2,LIDC-IDRI-0462,LIDC-IDRI-DRR-CROPPEDLIDC-IDRI-0462.png,['None'],['None'],['None'],6,3,3,NaN,NaN
3,LIDC-IDRI-0037,LIDC-IDRI-XRAY-CROPPEDLIDC-IDRI-0037.png,['Lung nodules'],['Covid-19 lesions'],['Covid-19 lesions'],3,2,1,NaN,NaN
4,LIDC-IDRI-0012,LIDC-IDRI-XRAY-CROPPEDLIDC-IDRI-0012.png,['Lung nodules'],['Lung nodules'],['Lung nodules'],16,12,4,NaN,NaN
5,LIDC-IDRI-0023,LIDC-IDRI-XRAY-CROPPEDLIDC-IDRI-0023.png,['None'],['None'],['None'],2,1,1,NaN,NaN
6,LIDC-IDRI-0529,LIDC-IDRI-DRR-CROPPEDLIDC-IDRI-0529.png,['None'],['None'],['None'],5,4,1,NaN,NaN
7,LIDC-IDRI-0003,LIDC-IDRI-XRAY-CROPPEDLIDC-IDRI-0003.png,['Lung nodules'],['Lung nodules'],['Lung nodules'],4,4,0,NaN,"**total number of lesions that received a ""nod..."
8,LIDC-IDRI-0308,LIDC-IDRI-DRR-CROPPEDLIDC-IDRI-0308.png,['None'],['None'],['None'],9,3,6,NaN,NaN
9,LIDC-IDRI-0376,LIDC-IDRI-DRR-CROPPEDLIDC-IDRI-0376.png,['None'],['None'],['None'],2,2,0,NaN,NaN


In [29]:
nodule_sizes["TCIA Patient ID"]

,TCIA Patent ID,Total Number of Nodules*,Number of Nodules >=3mm**,Number of Nodules <3mm***,Unnamed: 4,Unnamed: 5
0,LIDC-IDRI-0001,4,1,3,NaN,NaN
1,LIDC-IDRI-0002,12,1,11,NaN,*total number of lesions that received either ...
2,LIDC-IDRI-0003,4,4,0,NaN,"**total number of lesions that received a ""nod..."
3,LIDC-IDRI-0004,4,1,3,NaN,"***total number of lesions that received a ""no..."
4,LIDC-IDRI-0005,9,3,6,NaN,NaN
...,...,...,...,...,...,...
1014,LIDC-IDRI-1009,2,1,1,NaN,NaN
1015,LIDC-IDRI-1010,10,1,9,NaN,NaN
1016,LIDC-IDRI-1011,4,4,0,NaN,NaN
1017,LIDC-IDRI-1012,1,1,0,NaN,NaN


In [23]:
xray_df = df_metadata[df_metadata["Modality"]=="DX"].reset_index(drop=True)

In [27]:
for i in range(len(xray_df)):
    xray_files = [i for i in os.listdir(base_path + data_path + xray_df.iloc[i]["SeriesInstanceUID"]) if i.endswith(".dcm")]

    ds =  pydicom.dcmread(base_path + data_path+ xray_df.iloc[i]["SeriesInstanceUID"] +"/"+ xray_files[0])

    new_image = ds.pixel_array.astype(float)
    scaled_image = (np.maximum(new_image, 0) / new_image.max()) * 255.0
    scaled_image = np.uint8(scaled_image)
    final_image = Image.fromarray(scaled_image)
    

    final_image.save(base_path+output_path+xray_df.iloc[i]["PatientID"]+".png")